In [1]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import time
import tweepy
import numpy as np

In [3]:
def twitter_config():
    """
    Fungsi utilitas untuk mengkonfigurasi konsumsi file API Twitter dengan␣
    ,!kunci yang disediakan.
    """
    # Otentikasi dan akses menggunakan kunci:
    auth = tweepy.OAuthHandler("JLGvLiMHnOZkSDyfvmNZOwB7j","crxgjo1KIRh0HAWZNec1vKO7MlAdc1MX7XzGGgVgBA6Mt30GrB")
    auth.set_access_token("1572483046141751298-Q3ZqiGCrM8zcGcw0fepHL1luRLXXr6","A0mYVIrOVVc2KHDdol3hsGS0LFFXiRPfxToGk7FIBWt97")
    # Kembalikan akses ke API:
    api = tweepy.API(auth)
    try:
        api.verify_credentials()
        print("Authentication OK")
    except:
        print("Error during authentication")
    return api

In [4]:
# buat extractor object
extractor = twitter_config()

Authentication OK


In [7]:
tweets = extractor.user_timeline(screen_name="ivanlanin", count=100)
print("Tweets terambil: {}.\n".format(len(tweets)))

Tweets terambil: 100.



In [9]:
# Kita dapat membuat kerangka data sebagai berikut:
dataset = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# Kami membuat tampilan kerangka data:
display(dataset.head(10))

,Tweets
0,@k__nip @narabahasa Iya. Ini di buku Tata Baha...
1,@cahya_chan2 @narabahasa Tanda garis miring di...
2,@curcumint Dengan senang hati. Kadang kita per...
3,@k__nip @narabahasa Mungkin. Dalam karya Melay...
4,"@akarenuja @Husen_Jafar Mungkin karena ""belum ..."
5,@peddyeros @curcumint Artikel @klinikhukum ini...
6,@Debby_CK Tidak.\n\nhttps://t.co/bLufSOkJaj
7,@curcumint @peddyeros Mungkin ini dapat dipaka...
8,@RulyFauzi @narabahasa 👏🏻
9,@RivaiRivai @narabahasa Tentu bukan. 😁


In [10]:
data = dataset

In [11]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
data['Tweets'] = data['Tweets'].str.lower()

print('Case Folding Result : \n')
print(data['Tweets'].head(5))
print('\n\n\n')

Case Folding Result : 

0    @k__nip @narabahasa iya. ini di buku tata baha...
1    @cahya_chan2 @narabahasa tanda garis miring di...
2    @curcumint dengan senang hati. kadang kita per...
3    @k__nip @narabahasa mungkin. dalam karya melay...
4    @akarenuja @husen_jafar mungkin karena "belum ...
Name: Tweets, dtype: object






In [12]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data['Tweets'] = data['Tweets'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data['Tweets'] = data['Tweets'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data['Tweets'] = data['Tweets'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data['Tweets'] = data['Tweets'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data['Tweets'] = data['Tweets'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data['Tweets'] = data['Tweets'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data['Text_tokens'] = data['Tweets'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data['Text_tokens'].head())
print('\n\n\n')

Tokenizing Result : 

0    [nip, iya, ini, di, buku, tata, bahasa, baku, ...
1    [chan, tanda, garis, miring, ditulis, tanpa, s...
2    [dengan, senang, hati, kadang, kita, perlu, me...
3    [nip, mungkin, dalam, karya, melayu, klasik, k...
4    [jafar, mungkin, karena, belum, kawin, memberi...
Name: Text_tokens, dtype: object






In [14]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

data['Text_tokens_fdist'] = data['Text_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(data['Text_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(bahasa, 2), (nip, 1), (iya, 1), (ini, 1), (d...
1    [(chan, 1), (tanda, 1), (garis, 1), (miring, 1...
2    [(dengan, 1), (senang, 1), (hati, 1), (kadang,...
3    [(nip, 1), (mungkin, 1), (dalam, 1), (karya, 1...
4    [(jafar, 1), (mungkin, 1), (karena, 1), (belum...
Name: Text_tokens_fdist, dtype: object


In [22]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data['Text_tokens_WSW'] = data['Text_tokens'].apply(stopwords_removal) 


print(data['Text_tokens_WSW'].head())

0    [nip, iya, buku, tata, bahasa, baku, bahasa, i...
1         [chan, tanda, garis, miring, ditulis, spasi]
2    [senang, hati, kadang, memutuskan, meneruskan,...
3              [nip, karya, melayu, klasik, ditemukan]
4    [jafar, kawin, harapan, sanggup, ditanggung, p...
Name: Text_tokens_WSW, dtype: object


In [23]:
data['Text_tokens_WSW']

0     [nip, iya, buku, tata, bahasa, baku, bahasa, i...
1          [chan, tanda, garis, miring, ditulis, spasi]
2     [senang, hati, kadang, memutuskan, meneruskan,...
3               [nip, karya, melayu, klasik, ditemukan]
4     [jafar, kawin, harapan, sanggup, ditanggung, p...
                            ...                        
95                                                   []
96    [iya, penunjuk, kekerabatan, diawali, huruf, k...
97                                                   []
98                                                   []
99                                [reboot, muat, ulang]
Name: Text_tokens_WSW, Length: 100, dtype: object

In [24]:
data.head()

,Tweets,Text_tokens,Text_tokens_fdist,Text_tokens_WSW
0,nip iya ini di buku tata bahasa baku bahasa in...,"[nip, iya, ini, di, buku, tata, bahasa, baku, ...","{'nip': 1, 'iya': 1, 'ini': 1, 'di': 1, 'buku'...","[nip, iya, buku, tata, bahasa, baku, bahasa, i..."
1,chan tanda garis miring ditulis tanpa spasi se...,"[chan, tanda, garis, miring, ditulis, tanpa, s...","{'chan': 1, 'tanda': 1, 'garis': 1, 'miring': ...","[chan, tanda, garis, miring, ditulis, spasi]"
2,dengan senang hati kadang kita perlu memutuska...,"[dengan, senang, hati, kadang, kita, perlu, me...","{'dengan': 1, 'senang': 1, 'hati': 1, 'kadang'...","[senang, hati, kadang, memutuskan, meneruskan,..."
3,nip mungkin dalam karya melayu klasik kata itu...,"[nip, mungkin, dalam, karya, melayu, klasik, k...","{'nip': 1, 'mungkin': 1, 'dalam': 1, 'karya': ...","[nip, karya, melayu, klasik, ditemukan]"
4,jafar mungkin karena belum kawin memberikan ha...,"[jafar, mungkin, karena, belum, kawin, memberi...","{'jafar': 1, 'mungkin': 1, 'karena': 1, 'belum...","[jafar, kawin, harapan, sanggup, ditanggung, p..."


In [33]:
pip install swifter

  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16322 sha256=1d6551a6b6cd1349975fca61553c7c3ef301dbb25b0a6d32114ed982b9ef573b
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2b\5e\f2\3931524f702ffd03309e96d35ee2fbf9c61c27377511ee8d4c
Successfully built swifter
Note: you may need to restart the kernel to use updated packages.


In [34]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

In [35]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data['Text_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['Text_tokens_stemmed'] = data['Text_tokens_WSW'].swifter.apply(get_stemmed_term)
print(data['Text_tokens_stemmed'])

271
------------------------
nip : nip
iya : iya
buku : buku
tata : tata
bahasa : bahasa
baku : baku
indonesia : indonesia
edisi : edisi
iv : iv
chan : chan
tanda : tanda
garis : garis
miring : miring
ditulis : tulis
spasi : spasi
senang : senang
hati : hati
kadang : kadang
memutuskan : putus
meneruskan : terus
perdebatan : debat
ketida : ketida
karya : karya
melayu : melayu
klasik : klasik
ditemukan : temu
jafar : jafar
kawin : kawin
harapan : harap
sanggup : sanggup
ditanggung : tanggung
pemerintah : perintah
artikel : artikel
dipakai : pakai
rujukan : rujuk
ck : ck
selamat : selamat
menulis : tulis
tesis : tesis
ketemu : ketemu
orangorang : orangorang
penutur : tutur
salah : salah
moniker : moniker
inggris : inggris
menyerap : serap
arab : arab
terlambat : lambat
dianalogikan : analogi
dunia : dunia
timur : timur
pakai : pakai
filsafat : filsafat
condong : condong
memilih : pilih
nama : nama
mirna : mirna
kr : kr
konsep : konsep
budaya : budaya
padanan : padan
na : na
perkembangan :

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

0     [nip, iya, buku, tata, bahasa, baku, bahasa, i...
1            [chan, tanda, garis, miring, tulis, spasi]
2     [senang, hati, kadang, putus, terus, debat, ke...
3                    [nip, karya, melayu, klasik, temu]
4     [jafar, kawin, harap, sanggup, tanggung, perin...
                            ...                        
95                                                   []
96    [iya, tunjuk, kerabat, awal, huruf, kapital, r...
97                                                   []
98                                                   []
99                                [reboot, muat, ulang]
Name: Text_tokens_stemmed, Length: 100, dtype: object


In [36]:
data.head()

,Tweets,Text_tokens,Text_tokens_fdist,Text_tokens_WSW,Text_tokens_stemmed
0,nip iya ini di buku tata bahasa baku bahasa in...,"[nip, iya, ini, di, buku, tata, bahasa, baku, ...","{'nip': 1, 'iya': 1, 'ini': 1, 'di': 1, 'buku'...","[nip, iya, buku, tata, bahasa, baku, bahasa, i...","[nip, iya, buku, tata, bahasa, baku, bahasa, i..."
1,chan tanda garis miring ditulis tanpa spasi se...,"[chan, tanda, garis, miring, ditulis, tanpa, s...","{'chan': 1, 'tanda': 1, 'garis': 1, 'miring': ...","[chan, tanda, garis, miring, ditulis, spasi]","[chan, tanda, garis, miring, tulis, spasi]"
2,dengan senang hati kadang kita perlu memutuska...,"[dengan, senang, hati, kadang, kita, perlu, me...","{'dengan': 1, 'senang': 1, 'hati': 1, 'kadang'...","[senang, hati, kadang, memutuskan, meneruskan,...","[senang, hati, kadang, putus, terus, debat, ke..."
3,nip mungkin dalam karya melayu klasik kata itu...,"[nip, mungkin, dalam, karya, melayu, klasik, k...","{'nip': 1, 'mungkin': 1, 'dalam': 1, 'karya': ...","[nip, karya, melayu, klasik, ditemukan]","[nip, karya, melayu, klasik, temu]"
4,jafar mungkin karena belum kawin memberikan ha...,"[jafar, mungkin, karena, belum, kawin, memberi...","{'jafar': 1, 'mungkin': 1, 'karena': 1, 'belum...","[jafar, kawin, harapan, sanggup, ditanggung, p...","[jafar, kawin, harap, sanggup, tanggung, perin..."


In [38]:
data.head(30)

,Tweets,Text_tokens,Text_tokens_fdist,Text_tokens_WSW,Text_tokens_stemmed
0,nip iya ini di buku tata bahasa baku bahasa in...,"[nip, iya, ini, di, buku, tata, bahasa, baku, ...","{'nip': 1, 'iya': 1, 'ini': 1, 'di': 1, 'buku'...","[nip, iya, buku, tata, bahasa, baku, bahasa, i...","[nip, iya, buku, tata, bahasa, baku, bahasa, i..."
1,chan tanda garis miring ditulis tanpa spasi se...,"[chan, tanda, garis, miring, ditulis, tanpa, s...","{'chan': 1, 'tanda': 1, 'garis': 1, 'miring': ...","[chan, tanda, garis, miring, ditulis, spasi]","[chan, tanda, garis, miring, tulis, spasi]"
2,dengan senang hati kadang kita perlu memutuska...,"[dengan, senang, hati, kadang, kita, perlu, me...","{'dengan': 1, 'senang': 1, 'hati': 1, 'kadang'...","[senang, hati, kadang, memutuskan, meneruskan,...","[senang, hati, kadang, putus, terus, debat, ke..."
3,nip mungkin dalam karya melayu klasik kata itu...,"[nip, mungkin, dalam, karya, melayu, klasik, k...","{'nip': 1, 'mungkin': 1, 'dalam': 1, 'karya': ...","[nip, karya, melayu, klasik, ditemukan]","[nip, karya, melayu, klasik, temu]"
4,jafar mungkin karena belum kawin memberikan ha...,"[jafar, mungkin, karena, belum, kawin, memberi...","{'jafar': 1, 'mungkin': 1, 'karena': 1, 'belum...","[jafar, kawin, harapan, sanggup, ditanggung, p...","[jafar, kawin, harap, sanggup, tanggung, perin..."
5,artikel ini mungkin dapat dipakai sebagai rujukan,"[artikel, ini, mungkin, dapat, dipakai, sebaga...","{'artikel': 1, 'ini': 1, 'mungkin': 1, 'dapat'...","[artikel, dipakai, rujukan]","[artikel, pakai, rujuk]"
6,ck tidak,"[ck, tidak]","{'ck': 1, 'tidak': 1}",[ck],[ck]
7,mungkin ini dapat dipakai sebagai rujukan,"[mungkin, ini, dapat, dipakai, sebagai, rujukan]","{'mungkin': 1, 'ini': 1, 'dapat': 1, 'dipakai'...","[dipakai, rujukan]","[pakai, rujuk]"
8,,[],{},[],[]
9,tentu bukan,"[tentu, bukan]","{'tentu': 1, 'bukan': 1}",[],[]


In [40]:
data['Tweets'].isnull().values.any()

False